## **RLHF usando PPO**


Entrenaremos dos modelos de lenguaje, uno optimista y otro pesimista para simular distintos tonos en atención al cliente, usando como recompensa un clasificador de sentimiento entrenado con el dataset IMDb. Emplearemos el **Aprendizaje por Refuerzo (RL)** para que el agente (el LLM) aprenda, mediante prueba y error, a generar el texto que maximice la recompensa. 

Para ello usaremos el **Proximal Policy Optimization (PPO)**, un algoritmo estable y eficiente desarrollado por OpenAI que regula la magnitud de las actualizaciones de la política. A lo largo del cuaderno implementaremos y entrenaremos tu agente con PPO sobre reseñas de IMDb, adquiriendo la experiencia necesaria para aplicar RL en otros dominios. 

#### **Instalando librerías requeridas**





In [ ]:
!pip install datasets trl==0.11.0
!pip install --upgrade typing_extensions
!pip install matplotlib

#### Importación de las librería requeridas

*Se recomienda importar todas las bibliotecas necesarias en un mismo lugar (aquí):*



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer,AutoModelForCausalLM
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
import os

import tarfile
import pickle
import json
import matplotlib.pyplot as plt
import torch
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn

#### **Definiendo funciones auxiliares**


In [ ]:
def save_to_json(data, file_path):
    """
    Guarda un diccionario en un archivo JSON.

    Args:
        data (dict): Diccionario que se va a guardar.
        file_path (str): Ruta al archivo JSON.
    """
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    print(f"Datos guardados correctamente en {file_path}")
    
    
def load_from_json(file_path):
    """
    Carga datos desde un archivo JSON.
    Args:
        file_path (str): Ruta al archivo JSON.

    Returns:
        dict: Datos cargados desde el archivo JSON.
    """
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data


In [ ]:
def pad_sequence_to_length(tensor, length, pad_token_id):
    """
    Rellena una secuencia (tensor 1D) hasta una longitud especificada.

    Args:
        tensor (torch.Tensor): Tensor unidimensional a rellenar.
        length (int): Longitud objetivo de la secuencia.
        pad_token_id (int): ID del token de relleno.

    Returns:
        torch.Tensor: Tensor rellenado si era más corto que `length`, o el tensor original si ya era igual o más largo.
    """
    padding_length = length - tensor.size(0)
    if padding_length > 0:
        padding = torch.full(
            (padding_length,),
            pad_token_id,
            dtype=torch.long,
            device=tensor.device
        )
        return torch.cat((tensor, padding))
    return tensor


def pad_list_to_batch_size(tensors, batch_size, pad_token_id):
    """
    Rellena una lista de tensores para formar un lote del tamaño especificado.

    1. Calcula la longitud máxima de las secuencias en `tensors`.
    2. Rellena cada secuencia hasta esa longitud.
    3. Si hay menos tensores que `batch_size`, añade tensores de solo relleno.
    4. Devuelve una lista de longitud exactamente `batch_size`.

    Args:
        tensors (List[torch.Tensor]): Lista de tensores unidimensionales a agrupar.
        batch_size (int): Tamaño final del lote.
        pad_token_id (int): ID del token de relleno.

    Returns:
        List[torch.Tensor]: Lista de tensores rellenados con longitud `batch_size`.
    """
    # Determina la longitud máxima entre todas las secuencias
    max_length = max(t.size(0) for t in tensors)

    # Rellena cada secuencia hasta max_length
    padded_tensors = [
        pad_sequence_to_length(t, max_length, pad_token_id)
        for t in tensors
    ]

    # Añade tensores de solo relleno hasta completar batch_size
    while len(padded_tensors) < batch_size:
        padded_tensors.append(
            torch.full((max_length,), pad_token_id, dtype=torch.long, device=tensors[0].device)
        )

    # Trunca la lista si supera batch_size
    return padded_tensors[:batch_size]


In [ ]:
def print_ppo_stats(stats, related_to_objective=False):
    """
    Imprime estadísticas de entrenamiento PPO.

    Args:
        stats (dict): Diccionario con las estadísticas de PPO.
        related_to_objective (bool): Si es True, muestra estadísticas relacionadas con la función objetivo.
    """
    print("Estadísticas de entrenamiento PPO\n")

    if related_to_objective:
        print("Estadísticas del objetivo:")
        print(f"  Divergencia KL (objective/kl): {stats['objective/kl']}")
        print(f"  Coeficiente KL (objective/kl_coef): {stats['objective/kl_coef']}")
        print(f"  Entropía (objective/entropy): {stats['objective/entropy']}\n")
        
        print("Pérdidas PPO (relacionadas con la minimización de la función objetivo):")
        print(f"  Pérdida de política (ppo/loss/policy): {stats['ppo/loss/policy']}")
        print(f"  Pérdida de valor (ppo/loss/value): {stats['ppo/loss/value']}")
        print(f"  Pérdida total (ppo/loss/total): {stats['ppo/loss/total']}\n")
        
        print("Estadísticas de la Política PPO:")
        print(f"  Entropía de política (ppo/policy/entropy): {stats['ppo/policy/entropy']}")
        print(f"  KL aproximado (ppo/policy/approxkl): {stats['ppo/policy/approxkl']}")
        print(f"  Fracción de clip (ppo/policy/clipfrac): {stats['ppo/policy/clipfrac']}\n")
    else:
        print("Recompensa y estimación de la función de valor:")
        print(f"  Recompensa promedio sin puntaje (ppo/mean_non_score_reward): {stats['ppo/mean_non_score_reward']}")
        print(f"  Puntajes medios (ppo/mean_scores): {stats['ppo/mean_scores']}")
        print(f"  Desviación estándar de puntajes (ppo/std_scores): {stats['ppo/std_scores']}")
        print(f"  Predicción de valor (ppo/val/vpred): {stats['ppo/val/vpred']}")
        print(f"  Error de predicción de valor (ppo/val/error): {stats['ppo/val/error']}")
        print(f"  Varianza de predicción de valor (ppo/val/var): {stats['ppo/val/var']}")
        print(f"  Media de predicción de valor (ppo/val/mean): {stats['ppo/val/mean']}")
        print(f"  Varianza explicada (ppo/val/var_explained): {stats['ppo/val/var_explained']}\n")
    
    print("Longitudes de tokens:")
    print(f"  Longitud media de consultas (tokens/queries_len_mean): {stats['tokens/queries_len_mean']}")
    print(f"  Longitud media de respuestas (tokens/responses_len_mean): {stats['tokens/responses_len_mean']}\n")
    
    print("Estadísticas de Ttempo:")
    print(f"  Tiempo total (time/ppo/total): {stats['time/ppo/total']} segundos\n")

# Ejemplo de uso con stats proporcionadas y la bandera related_to_objective


### **Inicialización de la configuración, el modelo y el tokenizador de PPO**




La clase `PPOConfig` se utiliza para especificar el modelo y la tasa de aprendizaje para el entrenamiento de PPO. En este caso, el modelo es `"lvwerra/gpt2-imdb"` y la tasa de aprendizaje se establece en `1.41e-5`.



In [ ]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5)

`config.model_name` se refiere al identificador del modelo que se usa en la configuración para cargar el modelo preentrenado. Especifica qué modelo cargar desde el repositorio de Hugging Face. En este caso, `config.model_name` está establecido como `"lvwerra/gpt2-imdb"`, lo que indica que se debe usar el modelo GPT-2 afinado en el conjunto de datos IMDb (por el usuario lvwerra). Este identificador es esencial para cargar la arquitectura y los pesos correctos durante el proceso de fine-tuning o inferencia.


In [ ]:
config.model_name

El diccionario `sent_kwargs` contiene parámetros para la canalización de análisis de sentimiento, especificando que se deben devolver todos los puntajes, que la función a aplicar es `"none"` y que el tamaño de lote es `2`.


In [ ]:
sent_kwargs = {"top_k":None, "function_to_apply": "none", "batch_size": 2}

La clase `AutoModelForCausalLMWithValueHead` se utiliza para cargar el modelo GPT-2 preentrenado con una cabeza de valor para el entrenamiento PPO. El modelo se carga a partir del nombre de modelo especificado en la configuración.

La clase `AutoTokenizer` se emplea para cargar el tokenizador correspondiente al modelo preentrenado. El token de relleno del tokenizador se establece como el token de fin de secuencia (EOS).


In [ ]:
model_1 = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Ignora la advertencia anterior, ya que la versión de `trl` que instalaste la maneja automáticamente.

In [ ]:
# Primer modelo
modelo = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

Durante el entrenamiento PPO se actualiza el modelo. Además, se utiliza un modelo de referencia para estabilizar la política mediante la divergencia de Kullback–Leibler (KL) entre la política actual y la política de referencia. La divergencia KL actúa como un término de regularización.



In [ ]:
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

### **Conjunto de datos y tokenización**

**Nombre del conjunto de datos:** IMDb

**Descripción:** El conjunto de datos IMDb consta de 50.000 reseñas de películas etiquetadas como "positive2 o "negative", indicando el sentimiento de cada reseña. Es comúnmente usado para tareas de análisis de sentimiento.

**Carga del conjunto de datos:**
Se carga usando la función `load_dataset` de la librería `datasets`, específicamente la división "train".

In [ ]:
dataset_name = "imdb"
ds = load_dataset(dataset_name, split = "train")

In [ ]:
N = 5
for sample in range(N):
    print('texto',ds[sample]['text'])
    print('etiqueta',ds[sample]['label'])


Renombra la columna `"text"` a `"review"`:


In [ ]:
ds = ds.rename_columns({"text": "review"})
ds


El conjunto de datos se filtra para incluir solo reseñas de más de 200 caracteres:


In [ ]:
ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

Usar un `LengthSampler` para muestrear diferentes longitudes de texto durante el procesamiento de datos introduce variabilidad, haciendo que el modelo sea más robusto y capaz de manejar entradas de longitud variable en escenarios reales. Este enfoque previene el sobreajuste al exponer al modelo a tamaños de entrada diversos, mejorando la generalización a datos nuevos. También asegura un entrenamiento eficiente al gestionar la longitud de los textos, manteniendo la practicidad y el rendimiento. En conjunto, `LengthSampler` potencia la adaptabilidad y efectividad del modelo al simular condiciones de entrenamiento realistas y variadas, con longitudes de muestra entre `input_min_text_length` e `input_max_text_length`.








In [ ]:
input_min_text_length, input_max_text_length = 2, 8

Crear un objeto `LengthSampler`:

In [ ]:
input_size = LengthSampler(input_min_text_length, input_max_text_length)
input_size

Este objeto `input_size`, instancia de `LengthSampler`, genera una longitud de texto aleatoria entre 2 y 8 en cada llamada:

In [ ]:
for i in range(10):
    size=input_size()
    print(f"La muestra {i} tiene longitud {size}\n")

Finalmente, necesitaremos muestrear tokens y obtener índices tokenizados. Verifiquemos este proceso con una muestra.



In [ ]:
sample=ds[0]
sample

Tokenizamos el texto de la clave `"review"` en IDs, truncamos la secuencia a la longitud deseada y la asignamos a `"input_ids"`:



In [ ]:
sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
sample["input_ids"]

Decodificamos esos IDs truncados de nuevo a texto y lo guardamos en `"query"`, ya que necesitaremos el texto plano para la función de recompensa:

In [ ]:
sample["query"] = tokenizer.decode(sample["input_ids"])
sample["query"] 


Para aplicar esto sobre todo el dataset, definimos la función `tokenize`, que combina tokenización, truncado y decodificación:


In [ ]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

Y la usamos con `map` para procesar todo el conjunto de datos, además de darle formato PyTorch:


In [ ]:
ds = ds.map(tokenize, batched=False)
ds.set_format(type="torch")

>Nota: puedes ignorar de forma segura la advertencia que aparece arriba.


In [ ]:
ds[0]

Podemos iterar y mostrar las primeras 5 muestras con su `'review'`, `'input_ids'` y `'query'`:

In [ ]:
for i, sample in enumerate(ds):
    if i >= 5:
        break
    print(f"Muestra {i+1}:")
    print(f"Revisión: {sample['review']}")
    print(f"IDs de entradas: {sample['input_ids']}")
    print(f"Consulta: {sample['query']}")
    print("-" * 50)

La función `build_dataset` integra todos estos pasos para crear el dataset que usará posteriormente `PPOTrainer`. Primero eliminamos cualquier variable previa y recargamos el dataset original:



In [ ]:
del(ds)
dataset_name="imdb"
ds = load_dataset(dataset_name, split="train")
ds = ds.rename_columns({"text": "review"})

In [ ]:
def build_dataset(config,
                  dataset_name="imdb",
                  input_min_text_length=2,
                  input_max_text_length=8,
                  tokenizer=tokenizer):
    """
    Construye y devuelve el dataset listo para PPOTrainer.

    Args:
        config:     Objeto con la configuración (incluye model_name).
        dataset_name (str): Nombre del dataset a cargar.
        input_min_text_length (int): Longitud mínima de entrada.
        input_max_text_length (int): Longitud máxima de entrada.
        tokenizer:  Tokenizador ya inicializado.

    Returns:
        Dataset: Objeto tokenizado y formateado para PyTorch.
    """
    # (Re)inicializa el tokenizador y su token de padding
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # Carga y filtra el dataset IMDb
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    # Crea  LengthSampler
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    # Función de tokenización y truncado
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"]     = tokenizer.decode(sample["input_ids"])
        return sample

    # Aplica tokenización y dar formato PyTorch
    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")

    return ds

Creamos el objeto dataset y verificamos su contenido:


In [ ]:
dataset = build_dataset(config)

Muestra la primera muestra con sus campos 'input_ids' y 'query'


In [ ]:
dataset[0]

#### **Función collator**

La función collator es crucial para preparar lotes de datos en un formato adecuado para el PPOTrainer. Se encarga de agrupar cada característica de las muestras de datos:


In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

La función collator se entiende mejor con un ejemplo. Podemos introducir dos muestras, cada una con `'input_ids'`, `'query'` y `'review'`:


In [ ]:
data = [
    {'input_ids': [1, 2, 3, 4], 'query': "texto de ejemplo", 'review': "Esta es una reseña de ejemplo."},
    {'input_ids': [5, 6, 7, 8], 'query': "otro ejemplo", 'review': "Otra reseña de ejemplo."}
]

Aplicamos la función collator a los datos anteriores:


In [ ]:
batch = collator(data)
batch

Ahora, `'input_ids'`, `'query'` y `'review'` contienen sus muestras correspondientes agrupadas en listas


### **Inicializa PPOTrainer**

Proximal Policy Optimization (PPO) es un algoritmo de aprendizaje por refuerzo especialmente adecuado para entrenar modelos generativos, incluidos los chatbots. Ayuda a resolver desafíos como mantener diálogos coherentes y contextualmente apropiados.

PPO mejora los métodos de gradiente de política para chatbots usando una función objetivo recortada, lo cual garantiza actualizaciones de política graduales y estables. Esto ayuda a mantener la calidad del diálogo. Los métodos tradicionales pueden presentar alta varianza e inestabilidad, resultando en comportamientos inconsistentes del chatbot. La región de confianza de PPO equilibra la exploración de nuevas respuestas y la explotación de las ya buenas, haciéndolo más fiable para entrenar chatbots.

El PPO Trainer recopila muestras de diálogo, optimiza la política del chatbot según estas muestras y gestiona los modelos de red neuronal. Esto asegura un entrenamiento estable y eficiente, llevando a respuestas de chatbot coherentes y de alta calidad.

Inicialicemos el PPOTrainer con la configuración y componentes especificados:

* `config`: Ajustes de configuración para el entrenamiento PPO, como tasa de aprendizaje y nombre del modelo.
* `modelo`: Modelo principal que se fine-tuneará con PPO.
* `tokenizer`: Tokenizador correspondiente al modelo, usado para procesar texto de entrada.
* `dataset`: Conjunto de datos para entrenamiento, que proporciona los datos de entrada al modelo.
* `data_collator`: Collator para manejar la creación de lotes y el formateo de los datos de entrada.


In [ ]:
ppo_trainer = PPOTrainer(config, modelo, ref_model, tokenizer, dataset=dataset, data_collator=collator)
print("El objeto ppo_trainer  ",ppo_trainer)

Ignora las advertencias anteriores, la versión de `trl` que instalaste soporta este módulo.


Determina el dispositivo adecuado (CPU o GPU) para el entrenamiento con el PPOTrainer:


In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  
print(device)

### **Función de recompensa**

En el aprendizaje por refuerzo con PPO, se usa una función de recompensa para proporcionar retroalimentación sobre la calidad de las acciones tomadas por la política. Para un modelo generativo como un chatbot, la función de recompensa puede evaluar la calidad de las respuestas generadas. A continuación se explica cómo usar el pipeline de análisis de sentimiento como función de recompensa:

El pipeline de análisis de sentimiento sirve para evaluar las respuestas del chatbot y asignarles una recompensa basada en la puntuación de sentimiento. El PPO Trainer optimiza la política del chatbot para generar respuestas con mejor recepción y más atractivas. Aunque no es un modelo de recompensa típico, permite entrenar al chatbot de forma sencilla y efectiva.

Primero, inicializa un pipeline de análisis de sentimiento usando un modelo preentrenado fine-tuneado con reseñas de IMDB. El modelo predice el sentimiento de entradas de texto, proporcionando puntuaciones para sentimientos positivos y negativos:

In [ ]:
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

Al evaluar un texto negativo, obtendrás algo así:


In [ ]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

La clave `score` representa la confianza del modelo en su predicción. Valores más altos indican mayor seguridad en la clasificación de sentimiento, ya sea "POSITIVE" o "NEGATIVE". Así, el valor para la clase `POSITIVE` puede emplearse para determinar los valores de recompensa. Por ejemplo, una puntuación alta en "POSITIVE" incrementa la recompensa. Por el contrario, si el modelo no está seguro de que una reseña sea positiva, produce una recompensa baja, reduciendo la recompensa total. Esto significa que las reseñas con sentimiento negativo disminuyen la recompensa global, mientras que las positivas la aumentan.



In [ ]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

### **Generación de respuestas usando PPO**

#### **Tokenización y preparación del lote de entrada**

Esta sección de código muestra cómo generar respuestas usando el PPO (Proximal Policy Optimization) Trainer. El proceso implica tokenizar la entrada, preparar el lote para el entrenamiento, generar respuestas y decodificar los tokens generados a texto legible.


El código primero recupera un lote de datos del dataloader del PPO Trainer y selecciona las dos primeras entradas para su procesamiento:


In [ ]:
batch = next(iter(ppo_trainer.dataloader))

El lote contiene las claves `label`, `input_ids` y `query`:


In [ ]:
batch.keys()

Ahora creamos un nuevo lote que contiene solo las dos primeras muestras del lote original:


In [ ]:
batch = {key: batch[key][0:2] for key in batch}
batch

Inicializamos una lista de `response_tensors` para almacenar las respuestas que luego se evaluarán:


In [ ]:
response_tensors = []

El siguiente código extrae los `input_ids` del `batch` y los asigna a `query_tensors`. Estos tensores representan las secuencias de entrada tokenizadas que el modelo usará para generar respuestas. Se llaman "query tensors" porque representan las consultas iniciales que el modelo procesará:


In [ ]:
query_tensors =  batch["input_ids"]
query_tensors

A continuación, definimos una función lambda `get_text` que toma una lista de respuestas (`response`) y decodifica cada tensor en la lista usando el tokenizador, convirtiéndolo de nuevo en texto legible. El método `squeeze()` elimina dimensiones de tamaño 1 del tensor:

In [ ]:
get_text = lambda response:''.join([tokenizer.decode(r.squeeze()) for r in response])

Podemos ver las consultas originales en su forma de texto:


In [ ]:
get_text(query_tensors)

El diccionario `generation_kwargs` establece los parámetros para generar una secuencia del LLM (Modelo de Lenguaje). Los parámetros son:

* `"min_length": -1` — Sin longitud mínima para el texto generado.
* `"top_k": 0.0` — Sin filtrado top-k de tokens más probables.
* `"top_p": 1.0` — Sin muestreo de núcleo (nucleus sampling), usando toda la distribución.
* `"do_sample": True` — Habilita muestreo, permitiendo respuestas variadas.
* `"pad_token_id": 50256` — ID del token de relleno, para uniformar la longitud de las secuencias.


In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": 50256,
}
generation_kwargs

El `output_length_sampler` se inicializa con `LengthSampler(output_min_length, output_max_length)`. Este objeto se utiliza para muestrear las longitudes de salida de las secuencias generadas, garantizando que se encuentren dentro del rango de longitud mínima y máxima especificado. Al variar las longitudes, se pueden generar resultados más diversos y naturales a partir del modelo de lenguaje, lo que evita la generación de secuencias demasiado cortas o excesivamente largas y mejora la calidad general de las respuestas.



In [ ]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

El código llama a `output_length_sampler` para determinar la longitud de las secuencias generadas. La longitud muestreada se almacena en la variable `gen_len`.


In [ ]:
gen_len = output_length_sampler()
gen_len 

A continuación, establecemos el parámetro `max_new_tokens` en `generation_kwargs` al valor de `gen_len`, asegurando que el número máximo de tokens nuevos esté dentro del rango deseado:

In [ ]:
generation_kwargs["max_new_tokens"] = gen_len
generation_kwargs

Ahora procesemos la primera muestra usando PPO. Empezamos extrayendo el primer tensor de consulta:


In [ ]:
query=query_tensors[0]
query

Generamos una respuesta para la consulta usando el PPO Trainer con los parámetros de generación especificados. El tensor de respuesta se almacena en `response`:


In [ ]:
response = ppo_trainer.generate(query, **generation_kwargs)
response 

> Nota: Puedes ignorar la advertencia anterior.

Podemos imprimir el texto decodificado de la consulta y la respuesta usando `get_text`, para mostrar cómo el modelo ha añadido texto a la consulta original:

In [ ]:
print("Consulta:",get_text(query))
print("respuesta:", get_text(response))


Finalmente, añadimos los tokens recién generados a la lista `response_tensors`. El método `squeeze()` elimina dimensiones de tamaño 1, y `[-gen_len:]` asegura que solo se incluyan los tokens generados en esta iteración

In [ ]:
response_tensors.append(response.squeeze()[-gen_len:])
print("tokens recién generados de la respuesta:", get_text(response_tensors[-1]))

Repetimos el proceso para la segunda muestra. Esta sección genera una respuesta para una consulta dada, decodifica la parte relevante y la añade a la lista `response_tensors`.

In [ ]:
query = query_tensors[1]
gen_len = output_length_sampler()
generation_kwargs["max_new_tokens"] = gen_len
response = ppo_trainer.generate(query, **generation_kwargs)
print("Consulta:", get_text(query))
print("response acumulada:", get_text(response_tensors))
response_tensors.append(response.squeeze()[-gen_len:])
print("tokens recién generados de la segunda respuesta:", get_text(response_tensors[-1]))

Convertimos cada tensor en `response_tensors` a texto legible y lo almacenamos en el diccionario `batch` bajo la clave `response`:

In [ ]:
batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
batch["response"]


El lote ahora contiene tanto `query` como `response`:


In [ ]:
batch

#### **Función de puntuación**

A continuación, preparamos los textos para el análisis de sentimiento, que puede formar parte de una función de recompensa en PPO. Extraemos las cadenas `query` y `response` y las combinamos:


In [ ]:
texts = [q + r for q, r in zip(batch["query"], batch["response"])]
texts

Las puntuaciones de sentimiento (`pipe_outputs`) se pueden utilizar como retroalimentación para actualizar la política



In [ ]:
pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
pipe_outputs

Estas puntuaciones permiten evaluar la calidad o relevancia de las respuestas generadas, lo que indica la confianza del modelo en la probabilidad de que sean positivas. Las puntuaciones de las respuestas generadas se extraen de la lista `pipe_outputs`. Cada elemento de `pipe_outputs` contiene una lista de puntuaciones correspondientes a la salida del modelo.



Esta línea itera sobre la lista `pipe_outputs`, extrae la puntuación de cada salida, la convierte en un tensor y la almacena en la lista `rewards`. Las puntuaciones representan la confianza del modelo en la probabilidad de que las respuestas sean oraciones positivas.


In [ ]:
positive_scores = [
    item["score"]
    for output in pipe_outputs
    for item in output
    if item["label"] == "POSITIVE"
]
rewards = [torch.tensor(score) for score in positive_scores]
rewards

### **Optimización de política proximal (PPO)**

El bucle de entrenamiento es responsable de realizar un único paso de actualización del algoritmo PPO. Las entradas a este proceso son los tensores de consulta, respuesta y recompensa.

In [ ]:
print("Consulta:", get_text(query_tensors))
print("\n")
print("respuesta:", get_text(response_tensors))

Para cumplir con el requisito de tamaño mínimo de lote (batch) de 128 del PPO Trainer, puedes rellenar (`pad`) los tensores de respuesta con muestras adicionales:

In [ ]:
batch_size=128
pad_token_id = tokenizer.pad_token_id

query_tensors = pad_list_to_batch_size(query_tensors, batch_size, pad_token_id)

response_tensors = pad_list_to_batch_size(response_tensors, batch_size, pad_token_id)
rewards=rewards+[torch.tensor(0) for _ in range(batch_size-len(rewards))]

Ahora llama al método `step` del PPO Trainer para actualizar el modelo usando el algoritmo PPO con `query_tensors`, `response_tensors` y `rewards`.

* Calcula las pérdidas de la política y de la función de valor.
* Computa los gradientes y actualiza los parámetros de la red de política para mejorarla.
* Garantiza que la actualización de la política se mantenga dentro de un rango determinado, evitando grandes desplazamientos de política, que es un aspecto central de PPO.



> **Nota:** El siguiente código está comentado para evitar que el kernel falle por no tener GPU en el entorno actual. Para ejecutarlo, descarga el notebook y ejecútalo en un entorno con GPU. Descomenta el código antes de ejecutarlo.



In [ ]:
# stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

La variable `stats` es un diccionario que contiene diversas estadísticas del paso de entrenamiento PPO. Puedes imprimir sus claves usando la función `print_ppo_stats`. Estas estadísticas se organizan en dos categorías principales:

* **Minimización de la pérdida del modelo de lenguaje** (`related_to_objective=True`):
  Incluye métricas relacionadas con la optimización de parámetros del modelo, como la pérdida de política y la pérdida de valor.
* **Cálculo de la recompensa**:
  Incluye métricas propias del aprendizaje por refuerzo, como estimaciones de ventaja y cálculos de recompensa.


In [ ]:
# stats.keys()

In [ ]:
# print_ppo_stats(stats, related_to_objective = True)

In [ ]:
# print_ppo_stats(stats)

In [ ]:
all_stats = []

La variable `sentiment` debe establecerse en `"NEGATIVE"` para respuestas malas y en `"POSITIVE"` para respuestas buenas:


In [ ]:
sentiment = "POSITIVE"

Este fragmento de código representa un bucle de entrenamiento para el algoritmo PPO utilizando análisis de sentimiento. El bucle itera sobre lotes de datos del dataloader de `ppo_trainer` y realiza los siguientes pasos:

1. **Extrae tensores de consulta**:
   Se obtienen los `input_ids` (tensores de consulta) del lote.

2. **Genera respuestas**:
   Para cada tensor de consulta, se genera una respuesta con `ppo_trainer.generate` usando los `generation_kwargs` especificados. Las respuestas se decodifican y se añaden al lote bajo la clave `response`.

3. **Calcula puntuaciones de sentimiento**:

   * Se preparan los textos concatenando consultas y respuestas.
   * Se realiza el análisis de sentimiento sobre los textos combinados para obtener las puntuaciones.
   * Se convierten las puntuaciones en tensores y se almacenan en la lista `rewards`.

4. **Ejecuta el paso de PPO**:

   * Se llama a `ppo_trainer.step` para actualizar el modelo usando PPO con los tensores de consulta, respuesta y las recompensas calculadas.
   * Este paso calcula pérdidas de política y de valor, computa gradientes y actualiza los parámetros de la red de política.
   * La actualización se limita para evitar cambios drásticos en la política.

5. **Registra estadísticas**:

   * Las estadísticas resultantes del paso de entrenamiento PPO se registran y almacenan en la lista `all_stats`.

> **Nota:** Entrenar el modelo en CPU será muy lento. Para tu conveniencia, ya se ha preentrenado el modelo en GPU y lo hemos guardado. Puedes omitir la parte de entrenamiento y cargar el modelo guardado, o descomentar el bloque de código para entrenarlo tú mismo.

In [ ]:
# Bucle de entrenamiento PPO
# for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
#     query_tensors = batch["input_ids"]
#     print(f"epoca {epoch}")
# ## Obtiene respuestas de GPT-2
#     response_tensors = []
#     for query in query_tensors:
#         gen_len = output_length_sampler()
#         generation_kwargs["max_new_tokens"] = gen_len
#         response = ppo_trainer.generate(query, **generation_kwargs)
#         response_tensors.append(response.squeeze()[-gen_len:])
#     batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

#     texts = [q + r for q, r in zip(batch["query"], batch["response"])]
#     pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
#     positive_scores = [
#            item["score"]
#            for output in pipe_outputs
#            for item in output
#            if item["label"] == sentiment
#        ]
#    rewards = [torch.tensor(score) for score in positive_scores]

#     #### Corremos los pasos PPO
#     stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
#     ppo_trainer.log_stats(stats, batch, rewards)
    
#     all_stats.append(stats)

In [ ]:
# # Guardamos el modelo
# model_dir = "ppo-good"
# os.makedirs(model_dir, exist_ok=True)

# # Guardamos la configuracion del modelo y los pesos
# model_1.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/gSWo8GeztngSmzHpqX_RaQ/ppo-good.pkl
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/we8t5N-45dVq3VhxGwYRAg/ppo-good-tar.gz

In [ ]:
# Nombre del archivo
file_name = "ppo-good-tar.gz"

# Abre el archivo tar.gz
with tarfile.open(file_name, "r:gz") as tar:
    # Extrae todo el contenido en el directorio actual
    tar.extractall()

print("Extracción completada.")

In [ ]:
model_dir = "ppov3new1"
model_1 = AutoModelForCausalLMWithValueHead.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Carga estadísticas de entrenamiento
file_name = "ppo-good.pkl"
with open(file_name, 'rb') as f:
    all_stats = pickle.load(f)

model_1.to(device)


> Nota: Puedes ignorar con seguridad la advertencia anterior.


#### **Gráfica de la pérdida de entrenamiento PPO y recompensa media**

1. **Extracción de valores**:

   * `loss_values`: Valores de la pérdida total obtenidos de `all_stats`.
   * `reward_values`: Valores de la recompensa media obtenidos de `all_stats`.

2. **Gráfica de la función de pérdida**:

   * Gráfica de línea de la pérdida total a lo largo de las épocas.

3. **Gráfica de las recompensas**:

   * Gráfica de línea de la recompensa media a lo largo de las épocas.

4. **Mostrar las gráficas**:

   * Organizar y mostrar las gráficas usando `plt.tight_layout()` y `plt.show()`.

In [ ]:
loss_values = [stat['ppo/loss/total'] for stat in all_stats]
reward_values = [stat['ppo/mean_scores'] for stat in all_stats]

# Grafica la pérdida
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(loss_values, label='Pérdida Total', color='b')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Pérdida de Entrenamiento PPO a lo largo del tiempo')
plt.legend()
plt.grid(True)

# Grafica las recompensas
plt.subplot(2, 1, 2)
plt.plot(reward_values, label='Recompensa media', color='g')
plt.xlabel('Época')
plt.ylabel('Recompensa')
plt.title('Recompensa media de PPO a lo largo del tiempo')
plt.legend()
plt.grid(True)

# Muestra las gráficas
plt.tight_layout()
plt.show()

#### **Generación y análisis de texto con PPO y modelos de referencia**

**Configuración del dispositivo**:

* Determinar si CUDA está disponible y asignar el dispositivo correspondiente.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline_device = 0 if device.type == "cuda" else -1

**Función de generación de texto**:

* `generate_some_text(input_text, mi_model)`: Tokeniza el texto de entrada, genera una respuesta y la decodifica.

In [ ]:
gen_kwargs = {
    "min_length": -1,
    "max_new_tokens": 20,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}

def generate_some_text(input_text, mi_model):
    # Tokeniza el texto de entrada
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)
    generated_ids = mi_model.generate(input_ids, **gen_kwargs)

    # Decodifica el texto generado
    generated_text_ = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return generated_text_

**Generar texto con el modelo PPO**:

* Generamos texto usando el modelo entrenado con PPO.


In [ ]:
input_text = "Once upon a time in a land far"

generated_text=generate_some_text(input_text,model_1)
generated_text

**Análisis de sentimiento**:

* Analiza el sentimiento del texto generado usando `sentiment_pipe`.



In [ ]:
pipe_outputs = sentiment_pipe(generated_text, **sent_kwargs)
pipe_outputs

**Generar texto con el modelo de referencia**:

* Genera texto usando el modelo de referencia.

In [ ]:
generated_text = generate_some_text(input_text,ref_model)
generated_text

### **Comparación de los modelos PPO y de referencia**

1. **Parámetros de generación**:

   * Define `gen_kwargs` para la generación de texto.

2. **Preparar el lote**:

   * Obtiene una muestra de tamaño `bs` del conjunto de datos y extraer los tensores de consulta.

3. **Generar respuestas**:

   * Para cada tensor de consulta, generar respuestas usando tanto el modelo de referencia como el modelo PPO.

4. **Decodificar respuestas**:

   * Decodifica los tensores de respuesta en texto legible.

5. **Calcular puntuaciones de sentimiento**:

   * Prepara textos concatenando consultas y respuestas.
   * Calcula las puntuaciones de sentimiento para las respuestas antes y después del entrenamiento usando `sentiment_pipe`.

6. **Almacenar resultados**:

   * Guarda consultas, respuestas y puntuaciones de sentimiento en `game_data`.
   * Convertir `game_data` a un DataFrame y devolverlo.

In [ ]:
def compare_models_on_dataset(modelo, ref_model, dataset, tokenizer, sentiment_pipe, sent_kwargs, device, output_length_sampler):
    gen_kwargs = {
        "min_length": -1, 
        "top_k": 0.0, 
        "top_p": 1.0, 
        "do_sample": True, 
        "pad_token_id": tokenizer.eos_token_id
    }
    
    bs = 16
    game_data = dict()
    dataset.set_format("pandas")
    df_batch = dataset[:].sample(bs)
    game_data["query"] = df_batch["query"].tolist()
    query_tensors = df_batch["input_ids"].tolist()

    response_tensors_ref, response_tensors = [], []

    #  Obtiene el máximo de embeddings de posición para ambos modelos
    max_position_embeddings_ref = ref_model.config.max_position_embeddings
    max_position_embeddings_model = modelo.config.max_position_embeddings

    for i in range(bs):
        gen_len = output_length_sampler()

        # Convierte tensores de consulta a input IDs
        input_ids = torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device)

        # Proceso para ref_model 
        total_length_ref = input_ids.shape[-1] + gen_len
        if total_length_ref > max_position_embeddings_ref:
            # Trunca input_ids para ajustarse al máximo
            max_input_length_ref = max_position_embeddings_ref - gen_len
            input_ids_ref = input_ids[:, -max_input_length_ref:]
            total_length_ref = input_ids_ref.shape[-1] + gen_len
        else:
            input_ids_ref = input_ids
        
        output = ref_model.generate(
            torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), 
            max_new_tokens=gen_len, 
            **gen_kwargs
        ).squeeze()[-gen_len:]
        response_tensors_ref.append(output)

        # Proceso para modelo
        total_length_model = input_ids.shape[-1] + gen_len
        if total_length_model > max_position_embeddings_model:
            max_input_length_model = max_position_embeddings_model - gen_len
            input_ids_model = input_ids[:, -max_input_length_model:]
            total_length_model = input_ids_model.shape[-1] + gen_len
        else:
            input_ids_model = input_ids
        
        output = modelo.generate(
            torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), 
            max_new_tokens=gen_len, 
            **gen_kwargs
        ).squeeze()[-gen_len:]
        response_tensors.append(output)

    game_data["respuesta (antes)"] = [tokenizer.decode(t) for t in response_tensors_ref]
    game_data["respuesta (después)"] = [tokenizer.decode(t) for t in response_tensors]

    texts_before = [q + r for q, r in zip(game_data["query"], game_data["respuesta (antes)"])]
    game_data["puntuaciones (antes)"] = [output[1]["score"] for output in sentiment_pipe(texts_before, **sent_kwargs)]

    texts_after = [q + r for q, r in zip(game_data["query"], game_data["respuesta (después)"])]
    game_data["puntuaciones (después)"] = [output[1]["score"] for output in sentiment_pipe(texts_after, **sent_kwargs)]

    df_results = pd.DataFrame(game_data)
    return df_results

In [ ]:
df_results = compare_models_on_dataset(model_1, ref_model, dataset, tokenizer, sentiment_pipe, sent_kwargs, device, output_length_sampler)
df_results

#### **Ejecución del modelo PPO con sentimiento negativo**

Este código ejecuta el bucle de entrenamiento PPO con el sentimiento configurado como **NEGATIVO**, lo que evalúa el desempeño del modelo cuando se priorizan las puntuaciones de sentimiento negativo. El bucle de entrenamiento genera respuestas, calcula las puntuaciones de sentimiento, actualiza el modelo y registra las estadísticas en cada época.


In [ ]:
sentiment = "NEGATIVE"

In [ ]:
# Bucle de entrenamiento PPO
# for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
#     query_tensors = batch["input_ids"]
#     print(f"época {epoch}")
#
#     #### Obtiene respuestas de GPT-2
#     response_tensors = []
#     for query in query_tensors:
#         gen_len = output_length_sampler()
#         generation_kwargs["max_new_tokens"] = gen_len
#         response = ppo_trainer.generate(query, **generation_kwargs)
#         response_tensors.append(response.squeeze()[-gen_len:])
#     batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
#
#     #### Calcula la puntuación de sentimiento
#     texts = [q + r for q, r in zip(batch["query"], batch["response"])]
#     pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
#     negative_scores = [
#         item["score"]
#         for output in pipe_outputs
#         for item in output
#         if item["label"] == sentiment
#     ]
#     rewards = [torch.tensor(score) for score in negative_scores]
#
#     #### Ejecuta el paso de PPO
#     stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
#     ppo_trainer.log_stats(stats, batch, rewards)
#    
#     all_stats.append(stats)

In [ ]:
# # Guarda el modelo entrenado
#
# model_dir = "ppo-bad"
# os.makedirs(model_dir, exist_ok=True)
#
# # Guarda configuración y pesos del modelo
# model_0.save_pretrained(model_dir)
# tokenizer.save_pretrained(model_dir)

**Nota:** Entrenar el modelo en CPU será muy lento. El modelo ya ha sido preentrenado usando GPU y guardado para tu conveniencia. Puedes omitir la parte de entrenamiento, continuar con el siguiente bloque de código y cargar el modelo guardado. Si deseas, puedes descomentar el bloque de entrenamiento anterior para entrenarlo tú mismo.

In [ ]:
# 1. Descarga los archivos con nombres claros
!wget -O ppo-bad.tar.gz \
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/8zCp__SHRSgGVlf5yP50Ag/ppo-bad-tar.gz
!wget -O ppo-bad.pkl \
  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/jMW99Z9mvxesgYR-H6y6Yw/ppo-bad.pkl


In [ ]:
import os
import tarfile
import pickle
import torch
from transformers import AutoTokenizer
from trl.models import AutoModelForCausalLMWithValueHead

# 1) Extraer el tar.gz en un directorio 'ppov3new_bad1'
tar_path = "ppo-bad.tar.gz"
extract_dir = "ppov3new_bad1"
if not os.path.isdir(extract_dir):
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(extract_dir)
    print(f"Contenido extraído en: {extract_dir}")

# 2) Función para encontrar la subcarpeta que contenga config.json
def find_model_dir(base):
    for entry in os.listdir(base):
        path = os.path.join(base, entry)
        if os.path.isdir(path) and "config.json" in os.listdir(path):
            return path
    # si no hay subcarpeta, asume que base ya tiene config.json
    return base

model_dir = find_model_dir(extract_dir)
print("Cargando modelo desde:", model_dir)

# 3) Cargar modelo y tokenizer con local_files_only
modelo = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_dir,
    local_files_only=True
)
tokenizer = AutoTokenizer.from_pretrained(
    model_dir,
    local_files_only=True
)

# 4) Mover al dispositivo adecuado
device = "cuda" if torch.cuda.is_available() else "cpu"
modelo.to(device)
print("Modelo cargado en:", device)

# 5) Cargar estadísticas de entrenamiento
with open("ppo-bad.pkl", "rb") as f:
    all_stats = pickle.load(f)
print("Estadísticas cargadas, número de epocas registrado:", len(all_stats))


In [ ]:
import os

extract_dir = "ppov3new_bad1"
for root, dirs, files in os.walk(extract_dir):
    if "config.json" in files:
        print("→ Encontrado config.json en:", root)


In [ ]:
import tarfile
model_dir = "ppov3new_bad1/ppov3new_bad1"
model_0 = AutoModelForCausalLMWithValueHead.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
# Carga estadísticas de entrenamiento
file_name = "ppo-bad.pkl"
with open(file_name, 'rb') as f:
    all_stats = pickle.load(f)

model_0.to(device)

>**Nota:** Puedes ignorar de forma segura la advertencia anterior.


### **Comparación de modelos con sentimiento negativo**

El siguiente código compara el desempeño del modelo PPO entrenado (`model_0`) y el modelo de referencia sobre el conjunto de datos dado. La función `compare_models_on_dataset` genera respuestas de ambos modelos, calcula sus puntuaciones de sentimiento y devuelve los resultados en un DataFrame (`df_results`). Esta comparación ayuda a evaluar qué tan bien el modelo PPO genera respuestas negativas cuando `sentiment` está en **NEGATIVO**.

Como el conjunto de datos es bastante grande, usaremos solo una muestra para la prueba.


In [ ]:
df_results = compare_models_on_dataset(model_0, ref_model, dataset, tokenizer, sentiment_pipe, sent_kwargs, device, output_length_sampler)
df_results

#### **Ejercicio: Comparando modelos PPO**

En este ejercicio, se pide comparar el desempeño de dos modelos PPO entrenados (`model_0` y `model_1`) usando la función `compare_models_on_dataset` y notarás la diferencia en su rendimiento.

**Pasos para comparar modelos**:

1. Llama a `compare_models_on_dataset` pasando `model_0` y `model_1`.
2. Visualiza el DataFrame resultante para analizar las diferencias.



In [ ]:
## Escribe tu codigo